In [ ]:
# !pip install qdrant-client -q

In [1]:
from qdrant_client import QdrantClient,models
import os
import pandas as pd
import numpy as np
client = QdrantClient(
    "https://b705e7ce-d268-4e54-96a2-0e98cf033bf2.us-east4-0.gcp.cloud.qdrant.io",
    api_key=os.getenv("QDRANT_API"),
)


In [2]:
# !pip install sentence-transformers


In [3]:
data = pd.read_csv("./light-novel-titles.csv")
data.head()

,titles,descriptions,genres,links
0,Thereafter of an Exiled Magician ~Somehow My E...,"Lain, a reincarnated person, lives his life as...","['Action', 'Adventure', 'Ecchi', 'Fantasy', 'H...",https://www.novelupdates.com/series/thereafter...
1,Mahoutsukai no Konyakusha (LN),I who was reincarnated into a world of sword a...,"['Action', 'Drama', 'Fantasy', 'Harem', 'Matur...",https://www.novelupdates.com/series/mahoutsuka...
2,Exiled Prince Without Skills: Infinite Growth ...,"Henry, born in a royal family without skills, ...","['Action', 'Adventure', 'Fantasy', 'Horror', '...",https://www.novelupdates.com/series/exiled-pri...
3,Reincarnation Of The Strongest Sword God,"Starting over once more, he has entered this “...","['Action', 'Fantasy', 'Martial Arts', 'School ...",https://www.novelupdates.com/series/reincarnat...
4,Hard Work Will Never Betray Me (LN),A youth whose effort wasn’t rewarded lost his ...,"['Action', 'Adventure', 'Fantasy']",https://www.novelupdates.com/series/hard-work-...


In [4]:
data.isna().sum()

titles          0
descriptions    0
genres          0
links           0
dtype: int64

In [5]:
data.shape

(1366, 4)

In [6]:
from sentence_transformers import SentenceTransformer
encoder = SentenceTransformer("all-MiniLM-L6-v2")

/home/varaprasad/miniconda3/envs/llama/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/varaprasad/miniconda3/envs/llama/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/varaprasad/miniconda3/envs/llama/lib/python3.10/site-packages/torch/cuda/__init__.py:118: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount

In [ ]:
collection = client.create_collection(collection_name="anime",vectors_config=models.VectorParams(
    size=encoder.get_sentence_embedding_dimension(),
    distance=models.Distance.COSINE))

In [ ]:
points=[models.PointStruct(id=idx,vector=encoder.encode(des[1]).tolist(),payload={"title":des[0],"description":des[1],"Genres":des[2],"Link":des[3]}) for idx,des in enumerate(data.values)]

In [ ]:
count=0

In [ ]:
# inserting data into Qdrant
# uploading 5 points at a time
# run this cell several times to upload all the data 
# loop is also not working so I tried this way 
# Ohh! I remembered there is time module,which can be used to run code after certain period, I can use it
import time
from tqdm import tqdm
for i in tqdm(range(235,1366,5)):
    client.upload_points(collection_name="anime",points=points[i:i+5])
    time.sleep(10)

In [ ]:
# Uploaded 321 points

In [94]:
# searching for similar anime
query = "ninja fighting"
query_vector = encoder.encode(query).tolist()
hits = client.search(collection_name="anime", query_vector=query_vector, limit=5)


In [96]:
for hit in hits:
    print(hit.payload["title"],"score:",hit.score)

Kono Sekai ga Game da to, Ore dake ga Shitte Iru (WN) score: 0.3846035
My Status as an Assassin Obviously Exceeds the Hero’s score: 0.37036106
Overkill Princess score: 0.34252304
Sign-in Marvel score: 0.3302602
Fairy Tale Chronicles score: 0.3204572
